In [6]:
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt

# Параметры
j = 12
N = 10  # количество точек - 1
h = 1 / N
alpha_j = 0.1 + 0.05 * j
n = 6   # количество базисных функций: φ_0 до φ_5 => многочлен 5-й степени

# Функция f(x)
def f(x):
    return alpha_j * np.exp(x) + (1 - alpha_j) * np.sin(x)

# Узлы интерполяции
x_vals = np.array([alpha_j + h * i for i in range(N + 1)])
y_vals = f(x_vals)

# Проверочные точки
x_star = x_vals[0] + (2 / 3) * h
x_star2 = x_vals[n // 2] +  h / 2
x_star3 = x_vals[-1] - h / 3

TEST_POINTS = [x_star, x_star2, x_star3]

def phi(i, x):
    return x ** i

def calculate_gram_matrix(n, x_vals):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i, j] = sum(phi(i, xk) * phi(j, xk) for xk in x_vals)
    return A

def calculate_beta(n, x_vals, y_vals):
    b = np.zeros(n)
    for i in range(n):
        b[i] = sum(fx * phi(i, xk) for fx, xk in zip(y_vals, x_vals))
    return b

def solve_system(A, b):
    return np.linalg.solve(A, b)

def approximate(alpha, x):
    return sum(alpha[i] * phi(i, x) for i in range(len(alpha)))

def calculate_error(alpha, x_vals):
    return np.sqrt(sum((f(xk) - approximate(alpha, xk)) ** 2 for xk in x_vals))

# Основная логика
A = calculate_gram_matrix(n, x_vals)
b = calculate_beta(n, x_vals, y_vals)
alpha = solve_system(A, b)

df_alpha = pd.DataFrame([ [round(a, 6) for a in alpha] ],
                        columns=[f"α{i}" for i in range(len(alpha))])

app_x_star = approximate(alpha, x_star)
app_x_star2 = approximate(alpha, x_star2)
app_x_star3 = approximate(alpha, x_star3)

df = pd.DataFrame({
    "Точка": ["x*", "x**", "x***"],
    "Значение x": [x_star, x_star2, x_star3],
    "f(x)": [f(x_star), f(x_star2), f(x_star3)],
    "φ(x)": [app_x_star, app_x_star2, app_x_star3]
})

# # Истинная ошибка
r_x_star = round(abs(f(x_star) - app_x_star),10)
r_x_star2 = round(abs(f(x_star2) - app_x_star2), 10)
r_x_star3 = round(abs(f(x_star3) - app_x_star3), 10)

error_bound = calculate_error(alpha, x_vals)

is_error_bound_valid = [
    r_x_star <= error_bound,
    r_x_star2 <= error_bound,
    r_x_star3 <= error_bound
]

error_table = pd.DataFrame({
    "Точка": ["x*", "x**", "x***"],
    "Значение x": [x_star, x_star2, x_star3],
    "r(ист)": [r_x_star, r_x_star2, r_x_star3],
    "Оценка погрешности Δ": [error_bound] * 3,
    "Неравенство выполняется?": is_error_bound_valid
})

# Отображение таблиц
display(df)
display(df_alpha)
display(error_table)


,Точка,Значение x,f(x),φ(x)
0,x*,0.766667,1.714927,1.714929
1,x**,1.050000,2.260583,2.260582
2,x***,1.666667,4.004765,4.004767


,α0,α1,α2,α3,α4,α5
0,0.694511,1.02948,0.286032,0.137965,-0.013215,0.020463


,Точка,Значение x,r(ист),Оценка погрешности Δ,Неравенство выполняется?
0,x*,0.766667,1.910800e-06,0.000003,True
1,x**,1.050000,4.747000e-07,0.000003,True
2,x***,1.666667,1.665900e-06,0.000003,True
